# Data Ops Challenge Main Analysis File
In this file, I'll be using a Jupyter notebook to produce the core visualizations that will supplement the presentation and be used for answering the provided guiding questions.

## Loading, Adjusting, and Merging the Data

In [60]:
#imports
import sqlite3
import pandas as pd
import plotly.graph_objects as plt
import plotly.express as px
from plotly.subplots import make_subplots

In [61]:
#first, i'm going to initially connect to the database
conn = sqlite3.connect("challenge.db")

#now i'll load in each table as a dataframe
cities_df = pd.read_sql("SELECT * FROM five_hundred_cities;", conn)
access_df = pd.read_sql("SELECT * FROM access;", conn)
variablelist_df = pd.read_sql("SELECT * FROM variable_list", conn)

In [62]:
#now, when reading through the tables, i saw that the FIPS value for the access table was NOT the standard 5 digits for some
#of the codes. after looking up the FIPS codes, i found out that any FIPS code that originally had a "0" at the beginning
#was now missing a 0 at the beginning. if it had a non-zero value at the beginning, it still had that value. so i'll be
#adding in a zero at the beginning of each FIPS code that was supposed to have a 0 in the access table
#this was actually the case in the 500 cities dataset as well, with the PlaceFIPS and TractFIPS codes so i fixed that too.
#i've included a print before and after to show that this was the case.

print(cities_df["TractFIPS"].head())
cities_df["TractFIPS"] = cities_df["TractFIPS"].astype(str).str.zfill(11).str[:5]
print(cities_df["TractFIPS"].head())

print(access_df["FIPS"].head(10))
access_df["FIPS"] = access_df["FIPS"].astype(str).str.zfill(5)
print(access_df["FIPS"].head(10))

0    1073000100
1    1073000300
2    1073000400
3    1073000500
4    1073000700
Name: TractFIPS, dtype: int64
0    01073
1    01073
2    01073
3    01073
4    01073
Name: TractFIPS, dtype: object
0    1001
1    1003
2    1005
3    1007
4    1009
5    1011
6    1013
7    1015
8    1017
9    1019
Name: FIPS, dtype: int64
0    01001
1    01003
2    01005
3    01007
4    01009
5    01011
6    01013
7    01015
8    01017
9    01019
Name: FIPS, dtype: object


In [63]:
#now, to make analysis easier since we want to consider both factors from the FDA food atlas and from the
#CDC 500 cities dataset, we're gonna merge the two tables. as i specified in the README.md, i noted that the
#first 5 digits of the tract FIPS code for the 500 cities dataset (not the city FIPS code for the 500 cities dataset)
#and the first 5 digits of the FIPS code for the FDA food atlas match 
#we also will extract lat/lon from geolocation "(lat, lon)" strings for visualization purposes later (specifically visualization 1)
cities_df[["lat","lon"]] = cities_df["Geolocation"].str.strip("()").str.split(",", expand=True)
cities_df["lat"] = cities_df["lat"].astype(float)
cities_df["lon"] = cities_df["lon"].astype(float)
merged = cities_df.merge(
    access_df,
    left_on="TractFIPS",   #county FIPS from cities in tract FIPS
    right_on="FIPS",       #padded county FIPS from the access table
    how="inner"            #only keep matches
)
print(merged.columns.tolist()) #making sure all the columns merged

['index_x', 'StateAbbr', 'PlaceName', 'PlaceFIPS', 'TractFIPS', 'Place_TractID', 'Population2010', 'ACCESS2_CrudePrev', 'ACCESS2_Crude95CI', 'ARTHRITIS_CrudePrev', 'ARTHRITIS_Crude95CI', 'BINGE_CrudePrev', 'BINGE_Crude95CI', 'BPHIGH_CrudePrev', 'BPHIGH_Crude95CI', 'BPMED_CrudePrev', 'BPMED_Crude95CI', 'CANCER_CrudePrev', 'CANCER_Crude95CI', 'CASTHMA_CrudePrev', 'CASTHMA_Crude95CI', 'CHD_CrudePrev', 'CHD_Crude95CI', 'CHECKUP_CrudePrev', 'CHECKUP_Crude95CI', 'CHOLSCREEN_CrudePrev', 'CHOLSCREEN_Crude95CI', 'COLON_SCREEN_CrudePrev', 'COLON_SCREEN_Crude95CI', 'COPD_CrudePrev', 'COPD_Crude95CI', 'COREM_CrudePrev', 'COREM_Crude95CI', 'COREW_CrudePrev', 'COREW_Crude95CI', 'CSMOKING_CrudePrev', 'CSMOKING_Crude95CI', 'DENTAL_CrudePrev', 'DENTAL_Crude95CI', 'DIABETES_CrudePrev', 'DIABETES_Crude95CI', 'HIGHCHOL_CrudePrev', 'HIGHCHOL_Crude95CI', 'KIDNEY_CrudePrev', 'KIDNEY_Crude95CI', 'LPA_CrudePrev', 'LPA_Crude95CI', 'MAMMOUSE_CrudePrev', 'MAMMOUSE_Crude95CI', 'MHLTH_CrudePrev', 'MHLTH_Crude95CI',

## Visualization 1 (the where)
This visualization aims to help supplement the guiding question **"where should we deploy a food access program?"**

We can find this out by first considering that the only table we have available for analysis from the FDA Food Access dataset is the "ACCESS" sub-table, which contains information about access and proximity to grocery stores with the following subcategories within that category:
- overall (population with low-access)
- household resources (low income & low access, households with no car & low access, SNAP households & low access)
- demographics (children, seniors, hispanic ethnicity, asian, american indian or alaska native, asian, hawaiian or pacific islander, multiracial with low access)

We should also consider that for the 500 cities dataset, though some of the columns regarding different health conditions and their prevalences only consider elderly populations (>= 65 years old or 50-74 years old) or only adult men or only adult women, generally it doesn't account for factors such as household resources or demographics but rather just for a general age range (18-64), it looks at "what is the crude prevalence of this health condition?"

Since we want to consider data from both datasets when we are considering where to deploy a food access program, we'll only consider the overall access to grocery stores (general population with low-access) from the FDA Food Access dataset so as to match the generality of the 500 cities dataset.

Though we have past data that we can use and a percent change we can consider with access to food in specific areas (change from 2010-2015), I will only be using the percentage of population impacted by low access to food in specific areas from the most recent data (2015) so that we can target where need is highest now. For an example on why we want to target this to answer the question of where we should deploy a food access program now, we can consider that if we have a town with a large change of overall low access to grocery stores of 3% to 6% from 2010 to 2015 and another town with no change of overall low access to grocery stores of 25% to 25% from 2010 to 2015, we would want to target the latter town for food access programs since, although they have didn't change, they are experiencing more food access challenges with more of their population now.

In a 2024 study by Gregory and Jones on health challenges related to household food security (https://www.ers.usda.gov/data-products/charts-of-note/chart-detail?chartId=108211), it was found that 5 chronic diseases increased their prevalence as household food security worsened. These diseases were **hypertension, arthritis, diabetes, asthma, and COPD**. The age range included was ages 19-64 working adults, with compared household food security ranging from high, marginal, low, and very low. I'll be using these 5 diseases as the target ones to look for in the 500 cities dataset related to low food access in the FDA food atlas dataset when trying to find locations to deploy a food access program.

In [64]:
#what are the columns that I want to analyze for this? (note: hypertension = long-term high blood pressure)
    #county, city, state for the actual location (County, PlaceName, State)
    #PCT_LACCESS_POP15 from the FDA food atlas dataset -> indicates population, low access to store (%), 2015
    #COPD_CrudePrev from the 500 cities dataset -> indicates the model-based estimate for crude prevalence of chronic obstructive pulmonary disease among adults aged >=18 years
    #CASTHMA_CrudePrev from the 500 cities dataset -> indicates the model-based estimate for crude prevalence of current asthma among adults aged >=18 years
    #DIABETES_CrudePrev from the 500 cities dataset -> indicates the model-based estimate for crude prevalence of diagnosed diabetes among adults aged >=18 years
    #ARTHRITIS_CrudePrev from the 500 cities dataset -> indicates the model-based estimate for crude prevalence of arthritis among adults aged >=18 years
    #BPHIGH_CrudePrev from the 500 cities dataset -> indicates the model-based estimate for crude prevalence of high blood pressure among adults aged >=18 years

#first, what we'll do is calculate a "priority score" for each county that we have available in our merged dataset.
#this will be calculated as the following:
    #mean(disease scores for copd, asthma, diabetes, arthritis, and hypertension) * low access to store % for that county
#the higher the score -> the more priority we should designate to that county to designate resources.
#why does this work? -> multiplication highlights counties with both high food access problems AND high disease prevalence

#creating a copy of the merged df for adding in the calculatuions of disease avg and priority score for each location
vis1_data = merged.copy() 
vis1_data["disease_avg"] = vis1_data[["COPD_CrudePrev", "CASTHMA_CrudePrev", "DIABETES_CrudePrev","ARTHRITIS_CrudePrev", "BPHIGH_CrudePrev"]].mean(axis=1)
vis1_data["priority_score"] = vis1_data["PCT_LACCESS_POP15"] * vis1_data["disease_avg"]

#now we aggregate the data on a county-level and state-level for the purposes of the visualization.
county_data = vis1_data.groupby(["FIPS", "County", "State"]).agg(
    priority_score=("priority_score", "mean"), #average priority score across all tracts in that county
    lat=("lat", "mean"), #average the centroid coordinates
    lon=("lon", "mean") #average the centroid coordinates
).reset_index() #make it into a normal df

state_data = county_data.groupby("State").agg(
    priority_score=("priority_score", "mean") #average priority score across all counties in that state
).reset_index() #make it into a normal df

#we also normalize the data to be on a 0-100 scale for the priority score so the visualization is easier to understand
min_val = state_data["priority_score"].min()
max_val = state_data["priority_score"].max()
state_data["priority_score_norm"] = ((state_data["priority_score"] - min_val) / (max_val - min_val) * 100)

#we get the top 10 highest priority score counties overall (not per state) to display on the map
#as the areas we should focus on
top10 = county_data.nlargest(10, "priority_score").copy()
#need to normalize data to be on 0-100 scale for the top 10 counties as well (like we did at the state level)
min_val_c = county_data["priority_score"].min()
max_val_c = county_data["priority_score"].max()
top10["priority_score_norm"] = ((top10["priority_score"] - min_val_c) / (max_val_c - min_val_c) * 100)
top10 = top10.sort_values("priority_score", ascending=False).reset_index(drop=True) #sort the values in descending order

#adding labels to the top 10 counties: rank, county, state and priority score
top10["RankLabel"] = (
    (top10.index+1).astype(str) + ". " +
    top10["County"] + ", " +
    top10["State"] +
    " (" + top10["priority_score_norm"].round(1).astype(str) + ")"
)

In [65]:
#here we'll actually generate the figure!
fig = plt.Figure()
#the background will be a state-level heatmap chloropleth
fig.add_trace(plt.Choropleth(
    locations=state_data["State"], #which geographic places are being colored
    z=state_data["priority_score_norm"], #numeric values used to determine color for each state (normalized priority score)
    locationmode="USA-states", #how to interpret locations field
    colorscale="RdBu_r",   #color scheme to shade the states
    colorbar_title="Priority Score (0–100)" #color bar title
))

#this overlays the top 10 counties we found before with markers and labels
fig.add_trace(plt.Scattergeo(
    lon=top10["lon"], #longitutde for each point
    lat=top10["lat"], #latitude for each point
    text=top10["RankLabel"], #label that appears with each marker (we defined it before above)
    mode="markers+text", #we draw a marker and text
    marker=dict(size=10, color="black"), #the marker is a medium sized black dot
    textposition="middle right", #where is the label relative to the marker
    textfont=dict(size=14, color="black", family="Arial Bold") #the text is larger than the dot, black in color, and font type is Arial bold
))

fig.update_geos(scope="usa") #tells plotly to show the usa
#this part gives the plot a title, updates how big it'll be, and also tells it to not give it a legend (it'll overlap w/ the bar)
fig.update_layout(title_text="State-Level Food Access Program Priority with Top 10 Counties Marked", height=750, showlegend=False)
fig.show() #show the figure


## Visualization 2 & 3 (the who)
This visualization aims to supplement the guiding questions **"how many people will be included? how many might be successfully engaged?"** and **"which subgroup of the population might benefit the most from the program?"**

In order to answer these questions, we can first and foremost take into account that we already have the top 10 counties & states where we should be deploying food access programs. According to the above visualization and the data given, those are:
1. Hamilton, TN
2. Dougherty, GA
3. Genesee, MI
4. Richmond, GA
5. Plymouth, MA
6. Brevard, FL
7. Osage, OK
8. Lake, IN
9. Harrison, MS
10. Caddo, LA

So, what we know for these areas is that their calculated "priority score", which is a multiplication of the percentage of the general population's low access to store as of 2015 and the crude prevalence of 5 chronic diseases who are most affected by limited food access in >= 18 year olds, is significantly high enough to be in this top 10. Therefore, these counties should be where the resources are deployed.

In order to first answer the question, **how many people will be included?**, we can use the "Population2010" column from the 500 cities dataset which provides the 2010 census population count for a specific city and state, and filter for these specific counties by the FIPS code.

Next, to answer the question **how many might be successfully engaged?** we should first consider the fact that the client is a national Medicare Advantage program, which is a form of health insurance. Therefore, any programs that would be deployed to help relieve food access would likely be targeting existing members of health insurance programs. In the 500 cities dataset, there is a column "ACCESS2_CrudePrev" that provides a model-based estimate for crude prevalence of current lack of health insurance among adults aged 18-64 years. As this is generally a percentage, for each county, we can take into consideration the 2010 census population count, and this percentage, and take the inverse (100 minus this value) to estimate the percentage with insurance coverage, who are more likely to be reachable and eligible for program outreach.

Last, to answer the question **which subgroup of the population might benefit the most from the program?**, we can use the FDA food atlas dataset as it contains more specific information about demographics and the like for which demographic groups are disproportionately affected within these top 10 counties, and therefore most likely to benefit from interventions.

In [73]:
#population2010 column = total population affected
#for the subgroup part we have the following in the FDA food atlas for subgroups/demographics:
#household resources (low income & low access, households with no car & low access, SNAP households & low access)
#demographics (children, seniors, hispanic ethnicity, asian, american indian or alaska native, asian, hawaiian or pacific islander, multiracial with low access)
#we do have some missing information for 2010 percentage values, so we will have to use the 2015 values for
#these demographics.
#how can we calculate impact for these subgroups? -> well, in the FDA food atlas each of these percentages for these
#demographics indicates the % that had a LOW access to a grocery store
#so we could take all of the values within these top 10 counties (2015 counts)
#and find out which groups are most affected by low access to a grocery store in these 10 counties and therefore would
#benefit most from the program
#so for each subgroup, we calculate estimated affected individuals

#we create a new copy of the merged dataframe for the purposes for getting all of the columns we need (that aren't in the
#top10 dataframe we created earlier for visualization1)
vis2_data = merged.copy()
#getting the same sorted, merged by county data with inclusion of population, household and demographic low access to grocery store %
vis2_county = vis2_data.groupby(["FIPS", "County", "State"]).agg(
    population=("Population2010", "sum"),
    access2=("ACCESS2_CrudePrev", "mean"),
    low_income=("LACCESS_LOWI15", "sum"),   
    households_no_car=("LACCESS_HHNV15", "sum"),
    SNAP_household=("LACCESS_SNAP15", "sum"),
    children=("LACCESS_CHILD15", "sum"),
    seniors=("LACCESS_SENIORS15", "sum"),
    white=("LACCESS_WHITE15", "sum"),
    black=("LACCESS_BLACK15", "sum"),
    hispanic=("LACCESS_HISP15", "sum"),
    asian=("LACCESS_NHASIAN15", "sum"),
    american_indian_or_alaska_native=("LACCESS_NHNA15", "sum"),
    hawaiian_or_pacific_islander=("LACCESS_NHPI15", "sum"),
    multiracial=("LACCESS_MULTIR15", "sum")
).reset_index()

#we get only the top 10 counties (same as the ones plotted in visualization 1)
vis2_county = vis2_county[vis2_county["FIPS"].isin(top10["FIPS"])].copy()
#get the engaged population as the total population census * (100 - those that dont have healthcare) / 100 to include
#those that would likely be engaged with healthcare programs like this one
vis2_county["engaged"] = vis2_county["population"] * (100 - vis2_county["access2"]) / 100

#adding rank info to the vis2_county dataframe so we can order the barplot
rank_map = dict(zip(top10["FIPS"], range(1, len(top10)+1)))
vis2_county["Rank"] = vis2_county["FIPS"].map(rank_map)
vis2_county = vis2_county.sort_values("Rank")
county_labels = vis2_county["Rank"].astype(str) + ". " + vis2_county["County"] + ", " + vis2_county["State"]

#calculating subgroup affected numbers -> first we get all the column names
subgroup_cols = [
    "low_income", "households_no_car", "SNAP_household",
    "children", "seniors", "white", "black", "hispanic", "asian",
    "american_indian_or_alaska_native", "hawaiian_or_pacific_islander", "multiracial"
]

#melting for plotting purposes
subgroup_data = vis2_county.melt(
    id_vars=["County", "State", "population", "engaged"],
    value_vars=subgroup_cols,
    var_name="Subgroup",
    value_name="Affected"
)

#for plotting
subgroup_data["CountyState"] = subgroup_data["County"] + ", " + subgroup_data["State"]
label_map = {
    "low_income": "Low Income & Low Access",
    "households_no_car": "Households w/ No Car & Low Access",
    "SNAP_household": "SNAP Households & Low Access",
    "children": "Children & Low Access",
    "seniors": "Seniors & Low Access",
    "white": "White & Low Access",
    "black": "Black & Low Access",
    "hispanic": "Hispanic & Low Access",
    "asian": "Asian & Low Access",
    "american_indian_or_alaska_native": "American Indian/Alaska Native & Low Access",
    "hawaiian_or_pacific_islander": "Hawaiian/Pacific Islander & Low Access",
    "multiracial": "Multiracial & Low Access"
}
subgroup_data["Subgroup"] = subgroup_data["Subgroup"].map(label_map)


In [109]:
#initializing the figure with 2 subplots and 2 titles
fig = make_subplots(
    rows=1, cols=2,
    subplot_titles=(
        "Total Population vs Engaged (Top 10 Counties)",
        "Subgroups Most Affected (Combined Top 10)"
    ),
    specs=[[{"type": "bar"}, {"type": "domain"}]]  # second subplot is a pie
)

#the first subplot, which has the total populations for each county,state in blue
#and the total engaged populations (those who are not NOT on health insurance)
#in orange
fig.add_trace(
    plt.Bar(
        x=county_labels,
        y=vis2_county["population"],
        marker_color="steelblue",
        name="Total"
    ),
    row=1, col=1
)
fig.add_trace(
    plt.Bar(
        x=county_labels,
        y=vis2_county["engaged"],
        marker_color="darkorange",
        name="Engaged"
    ),
    row=1, col=1
)
fig.update_layout(barmode="group")

#the second subplot, which has the total affected members of each subgroup that were gathered from the FDA
#food atlas for the combined top 10 counties
subgroup_summary = subgroup_data.groupby("Subgroup")["Affected"].sum().reset_index()
#we plot the number of members of each affected subgroup for the combined top 10 counties in green
fig.add_trace(
    plt.Pie(
        labels=subgroup_summary["Subgroup"],
        values=subgroup_summary["Affected"],
        textinfo="label+percent",
        insidetextorientation="radial",
        marker=dict(colors=px.colors.qualitative.Set3),
        name="Subgroups",
        showlegend=False   
    ),
    row=1, col=2
)


#overall layout updating
fig.update_layout(
    height=600, width=1200,
    title_text="Who Would be Affected by Food Access Programs?",
    legend=dict(
        orientation="h",   
        x=0.2,           
        y=0.9,           
        xanchor="left",
        yanchor="bottom",
        font=dict(size=10),
        itemsizing="constant"
    ),
    xaxis_tickangle=-45
)

fig.show()

## Putting it all together: The Impact
This section aims to supplement the guiding question **"what is the projected impact of this program?"**